In [1]:
from separate_models.scene_shift_cat import scene_transformer as cat_model_pre
from separate_models.scene_shift_ori_col import scene_transformer as ori_model_pre
from separate_models.scene_shift_loc_col import scene_transformer as loc_model_pre
from separate_models.scene_shift_dim import scene_transformer as dim_model_pre
import json
import csv
import os
import pickle
import copy
import torch
import os.path as osp
import pandas as pd
import numpy as np
from scipy.spatial.transform import Rotation as R
import transforms3d
import trimesh
from tqdm import tqdm
import yaml
from transforms.scene import (
    SeqToTensor,
    Get_cat_shift_info,
    Padding_joint,
    Augment_rotation
)
from torchvision.transforms import Compose
from torch.utils.data import Subset
from datasets.suncg_shift_seperate_dataset_deepsynth import SUNCG_Dataset
import logging
from data.house import Room, Node
from data.top_down import TopDownView
from PIL import Image
from data.rendered import RenderedScene

In [2]:
logging.getLogger("trimesh").setLevel(logging.ERROR)

compare = True

mode = 'gen'
room_size_cap = [6.05, 4.05, 6.05]
collision_threshold = 1 #0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
room_type = 'bedroom'
adding_wall = True
run = 'floor_1'

In [3]:
path_dim = "configs/scene_shift_dim_config.yaml"

In [4]:
with open(path_dim) as f:
    cfg = yaml.safe_load(f)

In [6]:
out_path = cfg["test"]["log_dir"]
if compare == True:
    out_path = 'records/bedroom'

In [7]:
if not os.path.exists(out_path):
    os.makedirs(out_path)

In [8]:
IMG_OUT_DIR = "out_path/" + "compare_{room_type}/topdown_{run}/"
os.makedirs(osp.join(IMG_OUT_DIR, "gt"), exist_ok=True)
os.makedirs(osp.join(IMG_OUT_DIR, "preds"), exist_ok=True)